In [6]:
from os import listdir
from os.path import isfile, join

from bs4 import BeautifulSoup
import lxml

import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import requests

import nltk

In [54]:
import string

In [39]:
nltk.download('words')

[nltk_data] Downloading package words to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [13]:
article_files = [f for f in listdir('/home/ubuntu/newyorktimes_nlp/data/articles') if isfile(join('/home/ubuntu/newyorktimes_nlp/data/articles', f))]

In [23]:
articles_df = pd.DataFrame(columns=['articleID','byline','webURL','newDesk'])
for file in article_files:
    file = file.strip("'")
    df = pd.read_csv('/home/ubuntu/newyorktimes_nlp/data/articles/{}'.format(file),error_bad_lines=False, encoding = "ISO-8859-1")
    df = df[['articleID','byline','webURL','newDesk']]
    articles_df = articles_df.append(df)

In [27]:
articles_df = articles_df[articles_df['newDesk'].isin(['Arts&Leisure','BookReview','Business','Climate','Culture','Dining','Foreign','Games','Learning','National','Podcasts','RealEstate','Science','Sports','Travel','Washington'])]
articles_df.shape

(4462, 4)

In [28]:
article_urls = list(articles_df['webURL'])

In [84]:
def get_article_text(url):
    response = requests.get(url)   
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    text = ''
    for node in soup.findAll(class_="css-1ygdjhk evys1bk0"):
        article = "".join(node.findAll(text=True))
        text = text + article
    return text

In [87]:
for url in article_urls:
    text = get_article_text(url)
    articles_df.loc[articles_df.webURL == url, 'articleText'] = text

In [89]:
articles_df.to_csv("articles_df.csv")

In [60]:
df = pd.read_pickle("articles.pkl")

In [32]:
df.head()

,Unnamed: 0,articleID,byline,webURL,newDesk,articleText
0,0,58691a5795d0e039260788b9,By JENNIFER STEINHAUER,https://www.nytimes.com/2017/01/01/us/politics...,National,washington — the most powerful and ambitious r...
1,1,586967bf95d0e03926078915,By MARK LANDLER,https://www.nytimes.com/2017/01/01/world/asia/...,Foreign,washington — president obama’s advisers wrestl...
2,2,58698a1095d0e0392607894a,By CAITLIN LOVINGER,https://www.nytimes.com/2017/01/01/crosswords/...,Games,monday puzzle — and so the holiday season has ...
3,5,5869afd495d0e0392607896c,By MICHAEL POWELL,https://www.nytimes.com/2017/01/01/sports/bask...,Sports,more than years ago an american indian boy cl...
4,6,5869d08f95d0e03926078980,By MIKE HALE,https://www.nytimes.com/2017/01/01/arts/televi...,Culture,sunday brought us the first helenandnoah episo...


In [40]:
words = set(nltk.corpus.words.words())
def get_english(text):
    english = " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha())
    return english

In [1]:
import re
import string

def clean_text(text):
#     try:
    text = str(text)
    clean_text = re.sub(r'[^\w\s]','',text) # remove punctiation
    clean_text = clean_text.lower() 
    clean_text = get_english(clean_text)
    clean_text = ''.join([i for i in clean_text if not i.isdigit()])
    return clean_text
#     except:
#         return " "

In [67]:
df['articleText'] = df['articleText'].apply(lambda x: clean_text(x))

In [68]:
df.articleText.head()

0    the most powerful and ambitious congress in wi...
1    president with an intractable problem in the s...
2    puzzle and so the holiday season officially en...
3    more than ago an boy a mountain and across the...
4    brought us the first episode of the affair thi...
Name: articleText, dtype: object

In [69]:
import pickle

with open('articles.pkl', 'wb') as output_file:
    pickle.dump(df, output_file)